In [ ]:
"""
Author: Amruth Karun M V
Date: 19-Oct-2021
"""

import os
import pandas as pd
import numpy as np
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPool2D,
    AveragePooling2D, Flatten, GlobalAveragePooling2D,
    Dense, Dropout)
from keras.layers.merge import concatenate

from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline

TRAIN_PATH = "../input/covid19/"
EPOCHS = 100
BATCH_SIZE = 128
LEARNING_RATE = 0.001
INPUT_SIZE = (224, 224)

def load_data():
    """
    Loads input data from directory
    Arguments: None
    Returns: Train and val generator
    """
    
    train_datagen =  keras.preprocessing.image.ImageDataGenerator(validation_split=0.2) # set validation split

    train_generator = train_datagen.flow_from_directory(
        TRAIN_PATH,
        target_size=INPUT_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=False,
        class_mode='categorical',
        subset='training') # set as training data

    validation_generator = train_datagen.flow_from_directory(
        TRAIN_PATH, 
        target_size=INPUT_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=False,
        class_mode='categorical',
        subset='validation') # set as validation data
    
    return train_generator, validation_generator


def inception_module(x, filters_1x1, filters_3x3_reduce, filters_3x3, 
                     filters_5x5_reduce, filters_5x5,filters_pool_proj,
                     name=None): 
    """
    Represents an inception block
    Arguments:
        x                                     -- input
        filters_1x1                           -- number of filters of the 1x1 convolutional 
                                                 layer in the first path
        filters_3x3_reduce, filters_3x3       -- number of filters corresponding to the 1x1 
                                                 and 3x3 convolutional layers in the second path
        filters_5x5_reduce, filters_pool_proj -- number of filters corresponding to the 1x1 and 
                                                 5x5  convolutional layer in the third path
        filters_pool_proj                     -- number of filters of the 1x1 convolutional layer
    Returns: output layer
    """
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu')(x)
    
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu')(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu')(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu')(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu')(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    
    return output


def load_model():
    """
    Creates a keras GoogleNet model
    Arguments: None
    Returns: GoogleNet Model
    """
    
    input_layer = Input(shape=(224, 224, 3))

    x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2')(input_layer)
    x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
    x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
    x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
    x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)

    x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='inception_3a')

    x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='inception_3b')

    x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)

    x = inception_module(x,
                     filters_1x1=192,
                     filters_3x3_reduce=96,
                     filters_3x3=208,
                     filters_5x5_reduce=16,
                     filters_5x5=48,
                     filters_pool_proj=64,
                     name='inception_4a')


    x1 = AveragePooling2D((5, 5), strides=3)(x)
    x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
    x1 = Flatten()(x1)
    x1 = Dense(1024, activation='relu')(x1)
    x1 = Dropout(0.7)(x1)
    x1 = Dense(3, activation='softmax', name='auxilliary_output_1')(x1)

    x = inception_module(x,
                     filters_1x1=160,
                     filters_3x3_reduce=112,
                     filters_3x3=224,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4b')

    x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=256,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4c')

    x = inception_module(x,
                     filters_1x1=112,
                     filters_3x3_reduce=144,
                     filters_3x3=288,
                     filters_5x5_reduce=32,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4d')


    x2 = AveragePooling2D((5, 5), strides=3)(x)
    x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
    x2 = Flatten()(x2)
    x2 = Dense(1024, activation='relu')(x2)
    x2 = Dropout(0.7)(x2)
    x2 = Dense(3, activation='softmax', name='auxilliary_output_2')(x2)

    x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_4e')

    x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)

    x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5a')

    x = inception_module(x,
                     filters_1x1=384,
                     filters_3x3_reduce=192,
                     filters_3x3=384,
                     filters_5x5_reduce=48,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5b')

    x = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)

    x = Dropout(0.4)(x)

    x = Dense(3, activation='softmax', name='output')(x)
    model = Model(input_layer, [x, x1, x2], name='GoogLeNet')
    model.summary()
    
    opt = Adam(learning_rate=LEARNING_RATE)
    model.compile(loss = keras.losses.categorical_crossentropy, optimizer=opt, metrics=['accuracy'])
    return model


def plot_curves(history):
    """
    Plots loss and accuracy and loss plots for
    training and validation datasets
    Arguments: 
        history -- training history
    Returns: None
    """
   
    plt.plot(history.history['loss'], color='b', label="Training loss")
    plt.plot(history.history['val_loss'], color='r', label="Validation loss")
    plt.legend()
    plt.title('Training Loss VS Validation Loss')
    plt.show()
    
    plt.plot(history.history['output_accuracy'], color='b', label="Training accuracy")
    plt.plot(history.history['val_output_accuracy'], color='r',label="Validation accuracy")
    plt.title('Training Accuracy VS Validation Accuracy')
    plt.legend()
    plt.show()
    

def get_confusion_matrix(model, data_generator):
    """
    Calculates the accuracy and displays the 
    confusion matrix for the input data
    Arguments:
        model           -- trained model
        data_generator  -- input data generator
    Returns: None
    """
    
    predictions = model.predict(data_generator, BATCH_SIZE)
    y_pred = np.argmax(predictions[0], axis=1)
    y_true = data_generator.classes
    class_names = ['COVID', 'Normal', 'Pneumonia']
    
    print("Score =", model.evaluate(data_generator, batch_size=BATCH_SIZE))
    print("Accuracy  = ", metrics.accuracy_score(y_true, y_pred))
    cm = metrics.confusion_matrix(y_true, y_pred)
    metrics.ConfusionMatrixDisplay(cm, display_labels=class_names).plot(cmap=plt.cm.Blues,
                                                                       xticks_rotation='vertical')
    plt.show()

    
def train_model(train_generator, val_generator):
    """
    Trains GoogleNet model and saves the 
    trained weights to an H5 file.
    Arguments: 
        train_generator   -- train data generator
        val_generator     -- validation data generator
    Returns: Trained model
    """
    
    # Loads the model
    model = load_model()
    earlystop = keras.callbacks.EarlyStopping(patience=10)
    callbacks = [earlystop]
    
    history = model.fit(
        train_generator, 
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=val_generator,
        validation_steps=val_generator.samples//BATCH_SIZE,
        steps_per_epoch=train_generator.samples//BATCH_SIZE,
        callbacks=callbacks)
    
    plot_curves(history)
    model.save_weights("model.h5")
    print("Model saved successfully!")
    
    return model


In [ ]:
train_generator, val_generator = load_data()
model = train_model(train_generator, val_generator)

print("Confusion matrix for train data:")
get_confusion_matrix(model, train_generator)

print("Confusion matrix for val/test data:")
get_confusion_matrix(model, val_generator)